In [8]:
import os
import pathlib

import h5py
import uproot_methods
import awkward
import numpy as np # < 1.24.0
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## [参考にしたプログラムでの前処理の様子](https://github.com/hqucms/ParticleNet/blob/master/tf-keras/keras_train.ipynb)

In [4]:
import logging
logging.basicConfig(level=logging.DEBUG, format='[%(asctime)s] %(levelname)s: %(message)s')

In [5]:
def _transform(dataframe, start=0, stop=-1, jet_size=0.8):
    from collections import OrderedDict
    v = OrderedDict()

    df = dataframe.iloc[start:stop]
    def _col_list(prefix, max_particles=200):
        return ['%s_%d'%(prefix,i) for i in range(max_particles)]
    
    _px = df[_col_list('PX')].values
    _py = df[_col_list('PY')].values
    _pz = df[_col_list('PZ')].values
    _e = df[_col_list('E')].values
    
    mask = _e>0
    n_particles = np.sum(mask, axis=1)

    px = awkward.JaggedArray.fromcounts(n_particles, _px[mask])
    py = awkward.JaggedArray.fromcounts(n_particles, _py[mask])
    pz = awkward.JaggedArray.fromcounts(n_particles, _pz[mask])
    energy = awkward.JaggedArray.fromcounts(n_particles, _e[mask])

    p4 = uproot_methods.TLorentzVectorArray.from_cartesian(px, py, pz, energy)
    pt = p4.pt

    jet_p4 = p4.sum()

    # outputs
    _label = df['is_signal_new'].values
    v['label'] = np.stack((_label, 1-_label), axis=-1)
    v['train_val_test'] = df['ttv'].values
    
    v['jet_pt'] = jet_p4.pt
    v['jet_eta'] = jet_p4.eta
    v['jet_phi'] = jet_p4.phi
    v['jet_mass'] = jet_p4.mass
    v['n_parts'] = n_particles

    v['part_px'] = px
    v['part_py'] = py
    v['part_pz'] = pz
    v['part_energy'] = energy

    v['part_pt_log'] = np.log(pt)
    v['part_ptrel'] = pt/v['jet_pt']
    v['part_logptrel'] = np.log(v['part_ptrel'])

    v['part_e_log'] = np.log(energy)
    v['part_erel'] = energy/jet_p4.energy
    v['part_logerel'] = np.log(v['part_erel'])

    v['part_raw_etarel'] = (p4.eta - v['jet_eta'])
    _jet_etasign = np.sign(v['jet_eta'])
    _jet_etasign[_jet_etasign==0] = 1
    v['part_etarel'] = v['part_raw_etarel'] * _jet_etasign

    v['part_phirel'] = p4.delta_phi(jet_p4)
    v['part_deltaR'] = np.hypot(v['part_etarel'], v['part_phirel'])

    def _make_image(var_img, rec, n_pixels = 64, img_ranges = [[-0.8, 0.8], [-0.8, 0.8]]):
        wgt = rec[var_img]
        x = rec['part_etarel']
        y = rec['part_phirel']
        img = np.zeros(shape=(len(wgt), n_pixels, n_pixels))
        for i in range(len(wgt)):
            hist2d, xedges, yedges = np.histogram2d(x[i], y[i], bins=[n_pixels, n_pixels], range=img_ranges, weights=wgt[i])
            img[i] = hist2d
        return img

#     v['img'] = _make_image('part_ptrel', v)

    return v

In [6]:
def convert(source, destdir, basename, step=None, limit=None):
    df = pd.read_hdf(source, key='table')
    logging.info('Total events: %s' % str(df.shape[0]))
    if limit is not None:
        df = df.iloc[0:limit]
        logging.info('Restricting to the first %s events:' % str(df.shape[0]))
    if step is None:
        step = df.shape[0]
    idx=-1
    while True:
        idx+=1
        start=idx*step
        if start>=df.shape[0]: break
        if not os.path.exists(destdir):
            os.makedirs(destdir)
        output = os.path.join(destdir, '%s_%d.awkd'%(basename, idx))
        logging.info(output)
        if os.path.exists(output):
            logging.warning('... file already exist: continue ...')
            continue
        v=_transform(df, start=start, stop=start+step)
        awkward.save(output, v, mode='x')

In [7]:
srcDir = '/home/suzukiy/Downloads/t-quark-tagging-dataset'
destDir = "./processed"

In [9]:
# conver training file
convert(os.path.join(srcDir, 'train.h5'), destdir=destDir, basename='train_file')

[2023-01-22 20:31:23,907] INFO: Total events: 1211000
[2023-01-22 20:31:23,908] INFO: ./processed/train_file_0.awkd
/home/suzukiy/miniconda3/envs/weaver/lib/python3.10/site-packages/uproot_methods/classes/TLorentzVector.py:189: RuntimeWarning: invalid value encountered in sqrt
  return self._trymemo("mass", lambda self: self.awkward.numpy.sqrt(self.mag2))


In [10]:
# conver validation file
convert(os.path.join(srcDir, 'val.h5'), destdir=destDir, basename='val_file')

[2023-01-22 20:32:16,178] INFO: Total events: 403000
[2023-01-22 20:32:16,178] INFO: ./processed/val_file_0.awkd


In [11]:
# conver testing file
convert(os.path.join(srcDir, 'test.h5'), destdir=destDir, basename='test_file')

[2023-01-22 20:32:23,284] INFO: Total events: 404000
[2023-01-22 20:32:23,284] INFO: ./processed/test_file_0.awkd
/home/suzukiy/miniconda3/envs/weaver/lib/python3.10/site-packages/uproot_methods/classes/TLorentzVector.py:189: RuntimeWarning: invalid value encountered in sqrt
  return self._trymemo("mass", lambda self: self.awkward.numpy.sqrt(self.mag2))


In [19]:
with awkward.load(os.path.join(destDir, 'train_file_0.awkd')) as a:
    for k in a.keys():
        print(k, a[k].shape)

label (1211000, 2)
train_val_test (1211000,)
jet_pt (1211000,)
jet_eta (1211000,)
jet_phi (1211000,)
jet_mass (1211000,)
n_parts (1211000,)
part_px (1211000,)
part_py (1211000,)
part_pz (1211000,)
part_energy (1211000,)
part_pt_log (1211000,)
part_ptrel (1211000,)
part_logptrel (1211000,)
part_e_log (1211000,)
part_erel (1211000,)
part_logerel (1211000,)
part_raw_etarel (1211000,)
part_etarel (1211000,)
part_phirel (1211000,)
part_deltaR (1211000,)


In [22]:
with awkward.load(os.path.join(destDir, 'train_file_0.awkd')) as a:
    n_parts = a['n_parts']
    part_ptrel = a['part_ptrel']

In [24]:
part_ptrel # JaggedArray of arrays of floats, one per jet

<JaggedArray [[0.5440148 0.12141211 0.12092024 ... 0.0018517966 0.00082539115 0.00051214313] [0.2562643 0.13806675 0.08198755 ... 0.00084493106 0.00084028265 0.00070302136] [0.26822957 0.10070178 0.079407245 ... 0.0009618387 0.0008305976 0.000674688] ... [0.09885303 0.07326635 0.072109826 ... 0.0012147326 0.00089267385 0.00037475012] [0.23019442 0.19640091 0.08804418 ... 0.00048533437 0.00045946418 0.00036987552] [0.21753137 0.10661038 0.07977863 ... 0.00079060986 0.0007220333 0.00063161226]] at 0x7f7ff23334c0>

## PyTorchに向けた前処理

- 多分こんなことをしなくてもよくて、固定長の変数と、maskのarrayさえあれば大丈夫なはず。
- なので、TRorentzVectorの変換だけパクる